# Introduction to Data Mining

In [1]:
def create_multipleChoice_widget(description, options, correct_answer):
    if correct_answer not in options:
        options.append(correct_answer)
    
    correct_answer_index = options.index(correct_answer)
    
    radio_options = [(words, i) for i, words in enumerate(options)]
    alternativ = widgets.RadioButtons(
        options = radio_options,
        description = '',
        disabled = False
    )
    
    description_out = widgets.Output()
    with description_out:
        print(description)
        
    feedback_out = widgets.Output()

    def check_selection(b):
        a = int(alternativ.value)
        if a==correct_answer_index:
            s = '\x1b[6;30;42m' + "Riktig." + '\x1b[0m' +"\n" #green color
        else:
            s = '\x1b[5;30;41m' + "Feil. " + '\x1b[0m' +"\n" #red color
        with feedback_out:
            clear_output()
            print(s)
        return
    
    check = widgets.Button(description="submit")
    check.on_click(check_selection)
    
    
    return widgets.VBox([description_out, alternativ, check, feedback_out])
                     
Q1 = create_multipleChoice_widget('blablabla',['apple','banana','pear'],'pear')
Q2 = create_multipleChoice_widget('lalalalal',['cat','dog','mouse'],'dog')
Q3 = create_multipleChoice_widget('jajajajaj',['blue','white','red'],'white')
                     
display(Q1)
display(Q2)
display(Q3)

ERROR: Error in parse(text = x, srcfile = src): <text>:1:5: unexpected symbol
1: def create_multipleChoice_widget
        ^


## Welcome to the data mining portion of the course
- This part of the course moves away from studying free text and will instead focus on more sturctured data analysis. 
- From now on the notebooks will use a language called R which was specifically created for data analytics. 
- Like the previous sections the code is being provided for your reference and it is not necessary to understand the specifics in order to progress through the remainder of the course. 

### In this notebook we will be covering 
1. How to select variable columns from a data frame 
2. How to change data types
3. Replacing null numeric cells with the column mean
4. Replacing null categorical cells with a zero 
5. Replacing null cell using random forest imputation  

## Selecting variable columns from a data frame 

In [ ]:
#select variable columns from the training_v2 data frame into a new data frame called practice_csv
install.packages(“dplyr”)
library(dplyr)
practice_csv = select(training_v2, 4, 43, 36, 79, 61, 42, 123, 107, 115, 46, 28, 29, 5, 16, 6, 35, 20)
 
#create sepsis and cardiovascular diagnosis columns from the diagnosis column
practice_csv<- mutate(practice_csv, sepsis = ifelse(apache_2_diagnosis == '113', '1', '0'))
practice_csv<- mutate(practice_csv, cardiovascular_diagnosis = ifelse(apache_2_diagnosis == '114', '1', '0'))

#delete original diagnosis column
practice_csv = select(practice_csv, -17)

## Changing data types of the selected variables 

In [ ]:
#change categorical variables to factor data type 
practice_csv$hospital_death<- as.factor(practice_csv$hospital_death)
practice_csv$sepsis<- as.factor(practice_csv$sepsis)
practice_csv$cardiovascular_diagnosis<- as.factor(practice_csv$cardiovascular_diagnosis)
practice_csv$intubated_apache<- as.factor(practice_csv$intubated_apache)
practice_csv$gcs_eyes_apache<- as.factor(practice_csv$gcs_eyes_apache)
practice_csv$gcs_motor_apache<- as.factor(practice_csv$gcs_motor_apache)

#change to numeric variables to numeric data type 
practice_csv$temp_apache<- as.numeric(practice_csv$temp_apache)
practice_csv$map_apache<- as.numeric(practice_csv$map_apache)
practice_csv$h1_heartrate_max<- as.numeric(practice_csv$h1_heartrate_max)
practice_csv$d1_resprate_max<- as.numeric(practice_csv$d1_resprate_max)
practice_csv$d1_potassium_max<- as.numeric(practice_csv$d1_potassium_max)
practice_csv$d1_creatinine_max<- as.numeric(practice_csv$d1_creatinine_max)
practice_csv$d1_hematocrit_max<- as.numeric(practice_csv$d1_hematocrit_max)
practice_csv$sodium_apache<- as.numeric(practice_csv$sodium_apache)
practice_csv$wbc_apache<- as.numeric(practice_csv$wbc_apache)
practice_csv$age<- as.numeric(practice_csv$age)
practice_csv$pre_icu_los_days<- as.numeric(practice_csv$pre_icu_los_days)
practice_csv$bmi<- as.numeric(practice_csv$bmi)

#check all variable data types to make sure they are correct
str(practice_csv)

## Replacing null values using measures of center (mean, median, mode)  

In [ ]:
#replace some numeric variable nulls with the column mean 
practice_csv = transform(practice_csv, bmi = ifelse(is.na(bmi), mean(bmi, na.rm=TRUE), bmi))
practice_csv = transform(practice_csv, temp_apache = ifelse(is.na(temp_apache), mean(temp_apache, na.rm=TRUE), temp_apache))
practice_csv = transform(practice_csv, map_apache = ifelse(is.na(map_apache), mean(map_apache, na.rm=TRUE), map_apache))
practice_csv = transform(practice_csv, h1_heartrate_max = ifelse(is.na(h1_heartrate_max), mean(h1_heartrate_max, na.rm=TRUE), h1_heartrate_max))
practice_csv = transform(practice_csv, d1_resprate_max = ifelse(is.na(d1_resprate_max), mean(d1_resprate_max, na.rm=TRUE), d1_resprate_max))
practice_csv = transform(practice_csv, d1_potassium_max = ifelse(is.na(d1_potassium_max), mean(d1_potassium_max, na.rm=TRUE), d1_potassium_max))
practice_csv = transform(practice_csv, d1_creatinine_max = ifelse(is.na(d1_creatinine_max), mean(d1_creatinine_max, na.rm=TRUE), d1_creatinine_max))
practice_csv = transform(practice_csv, d1_hematocrit_max = ifelse(is.na(d1_hematocrit_max), mean(d1_hematocrit_max, na.rm=TRUE), d1_hematocrit_max))
practice_csv = transform(practice_csv, sodium_apache = ifelse(is.na(sodium_apache), mean(sodium_apache, na.rm=TRUE), sodium_apache))

#replace some categorical variable nulls with 0 (0 is the mode for these columns)
#gcs_motor_apache nulls replaced with a 1 (1 is the mode for this column) 
practice_csv$sepsis[is.na(practice_csv$sepsis)] <- 0
practice_csv$cardiovascular_diagnosis[is.na(practice_csv$cardiovascular_diagnosis)] <- 0
practice_csv$intubated_apache[is.na(practice_csv$intubated_apache)] <- 0
practice_csv$gcs_motor_apache[is.na(practice_csv$gcs_motor_apache)] <- 1

## Replacing null values using a random forest algorithm 

In [ ]:
#use a random forest algorithm to replace the remaining nulls
#random forest is non-parametric and works for categorical and numeric variables
#ntree is the number of trees, maxiter is the maximum number of iterations (repeats) 
#more trees and more iterations can increase accuracy but takes more time to run
install.packages("missForest")
install.packages(“randomForest”)
library(missForest)
library(randomForest)
set.seed(96) 
practice_csv.imp <- missForest(practice_csv, verbose = TRUE, maxiter = 3, ntree = 10)

#check imputed values
practice_csv.imp$ximp

#check imputation error
practice_csv.imp$OOBerror

#assign imputed data frame to a new data frame called t
t<- practice_csv.imp$ximp

## This concludes our introduction to data analytics 
- The following questions were addressed between this notebook and the complementary video lecture:
1. What is a CSV file and how is it loaded into R? 
2. What are some simple and more advanced methods that can be used to deal with nulls? 
3. How should present but highly unlikely data be dealt with?  


# Exporting CSV

This section just provides a space to export the cleaned data you've already cleaned. I made a new section so it wouldn't interfere with your other work. 

-Eric 